# Example api

In [1]:
from adelecv.api.logs import enable_logs
from adelecv.api.data.segmentations import SegmentationDataset
from logging import StreamHandler

In [2]:
enable_logs(StreamHandler())

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
from adelecv.api.config import Settings
from pathlib import Path

In [5]:
# you can change the path to save logs and weights
# Settings.update_tmp_path(Path('your_path'))

In [6]:
from adelecv.api.config import Settings

In [7]:
Settings.TMP_PATH

WindowsPath('tmp/cb0cb0afd3f84fb6b0d8299532ad8759')

In [8]:
from adelecv.api.data.segmentations.types import ImageMask

In [9]:
segm_dataset = SegmentationDataset(
    r'F:\dataset\ph2',
    ImageMask, # аннотация для наследника
    (256, 256),
    (0.7, 0.2, 0.1),
    16,
)

 100% |█████████████████| 200/200 [1.6s elapsed, 0s remaining, 125.9 samples/s]      


INFO - Split dataset train size: 140, valid size: 40, test size: 20
INFO - Creating a dataset
DEBUG - Dataset created with params, dataset dir: F:\dataset\ph2, classes: {0: 'background', 1: 'target'}, batch size: 16


In [10]:
from adelecv.api.models.segmentations import  get_encoders, get_pretrained_weights, get_models, get_torch_optimizers, get_losses, get_optimize_scores
from adelecv.api.optimize.segmentations import get_hp_optimizers
# создать отдельный модуль для получения возможных параметров

In [11]:
print('models: ', get_models())
print('encoders: ', get_encoders())
print('pretrained_weights', get_pretrained_weights())
print('optimizers', get_torch_optimizers())
print('losses', get_losses())
print('hp optimizers', get_hp_optimizers())
print('optimize scores', get_optimize_scores())

models:  ['Unet', 'UnetPlusPlus', 'MAnet', 'Linknet', 'FPN', 'PSPNet', 'DeepLabV3', 'DeepLabV3Plus', 'PAN']
encoders:  ['resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152', 'resnext50_32x4d', 'resnext101_32x4d', 'resnext101_32x8d', 'resnext101_32x16d', 'resnext101_32x32d', 'resnext101_32x48d', 'dpn68', 'dpn68b', 'dpn92', 'dpn98', 'dpn107', 'dpn131', 'vgg11', 'vgg11_bn', 'vgg13', 'vgg13_bn', 'vgg16', 'vgg16_bn', 'vgg19', 'vgg19_bn', 'senet154', 'se_resnet50', 'se_resnet101', 'se_resnet152', 'se_resnext50_32x4d', 'se_resnext101_32x4d', 'densenet121', 'densenet169', 'densenet201', 'densenet161', 'inceptionresnetv2', 'inceptionv4', 'efficientnet-b0', 'efficientnet-b1', 'efficientnet-b2', 'efficientnet-b3', 'efficientnet-b4', 'efficientnet-b5', 'efficientnet-b6', 'efficientnet-b7', 'mobilenet_v2', 'xception', 'timm-efficientnet-b0', 'timm-efficientnet-b1', 'timm-efficientnet-b2', 'timm-efficientnet-b3', 'timm-efficientnet-b4', 'timm-efficientnet-b5', 'timm-efficientnet-b6', 'timm-e

In [12]:
from adelecv.api.optimize.segmentations import HyperParamsSegmentation
from adelecv.api.optimize.segmentations import HPOptimizer

In [28]:
params = HyperParamsSegmentation(
    strategy='TPESampler',
    architectures=['UnetPlusPlus'],
    encoders=['mobilenet_v2'],
    pretrained_weights=['imagenet'],
    loss_fns=['JaccardLoss'],
    optimizers=['AdamW', 'RMSprop'],
    epoch_range=(5, 5),
    lr_range=(0.001, 0.003),
    optimize_score='loss',
)

In [29]:
hp_optimizer = HPOptimizer(
    hyper_params=params,
    num_trials=1,
    device='GPU',
    dataset=segm_dataset
)

DEBUG - Create hp optimizer with params: strategy: TPESampler num_trials: 1 num_classes: 2 device: cuda hps: {'strategy': 'TPESampler', 'architectures': ['UnetPlusPlus'], 'encoders': ['mobilenet_v2'], 'pretrained_weights': ['imagenet'], 'loss_fns': ['JaccardLoss'], 'optimizers': ['AdamW', 'RMSprop'], 'epoch_range': (5, 5), 'lr_range': (0.001, 0.003), 'optimize_score': 'loss'}


In [15]:
from adelecv.api.config import Settings

In [16]:
tensorboard_logs_path = Settings.TENSORBOARD_LOGS_PATH.as_posix()

In [17]:
tensorboard_logs_path

'tmp/cb0cb0afd3f84fb6b0d8299532ad8759/tensorboard'

In [18]:
%reload_ext tensorboard

In [19]:
tensorboard_logs_path

'tmp/cb0cb0afd3f84fb6b0d8299532ad8759/tensorboard'

In [22]:
tensorboard --logdir $tensorboard_logs_path

^C


Launching TensorBoard...

In [30]:
hp_optimizer.optimize()

INFO - Train models started
[I 2023-02-28 20:12:23,795] A new study created in memory with name: no-name-27ad1279-2653-4d1b-9e5b-422a70b86bd0
DEBUG - Create tensorboard logger, path: tmp/cb0cb0afd3f84fb6b0d8299532ad8759/tensorboard/0a8754a
DEBUG - Dataset updated
INFO - Model 0a8754a_UnetPlusPlus_mobilenet_v2_imagenet_AdamW_JaccardLoss_lr=0,002541036844019805 trained with test loss 0.1143430769443512
DEBUG - Add predictions for model: 0a8754a_UnetPlusPlus_mobilenet_v2_imagenet_AdamW_JaccardLoss_lr=0,002541036844019805


 200 [11.3s elapsed, 17.3 samples/s]  


DEBUG - Save weights model: 0a8754a_UnetPlusPlus_mobilenet_v2_imagenet_AdamW_JaccardLoss_lr=0,002541036844019805, path: tmp/cb0cb0afd3f84fb6b0d8299532ad8759/weights/0a8754a.pt
[I 2023-02-28 20:13:13,306] Trial 0 finished with value: 0.10235309600830078 and parameters: {'optimizer': 'AdamW', 'architecture': 'UnetPlusPlus', 'encoders': 'mobilenet_v2', 'pretrained_weight': 'imagenet', 'lr': 0.002541036844019805, 'loss': 'JaccardLoss', 'num_epoch': 5}. Best is trial 0 with value: 0.10235309600830078.
INFO - Study statistics:
INFO - Number of finished trials: 1
INFO - Number of pruned trials: 0
INFO - Number of complete trials: 1
INFO - Best trial:
INFO - Value: 0.10235309600830078
INFO - Params: 
INFO - optimizer: AdamW
INFO - architecture: UnetPlusPlus
INFO - encoders: mobilenet_v2
INFO - pretrained_weight: imagenet
INFO - lr: 0.002541036844019805
INFO - loss: JaccardLoss
INFO - num_epoch: 5
INFO - Train models is over


In [31]:
hp_optimizer.stats_models

,_id,name,architecture,encoder,pretrained_weight,lr,optimizer,loss_fn,num_epoch,loss,fbeta_score,f1_score,iou_score,accuracy,positive_predictive_value,sensitivity
0,0a8754a,0a8754a_UnetPlusPlus_mobilenet_v2_imagenet_Ada...,UnetPlusPlus,mobilenet_v2,imagenet,0.002541,AdamW,JaccardLoss,5,0.114,0.921,0.921,0.861,0.943,0.931,0.926


In [25]:
from adelecv.api.modification_models import ExportWeights

In [ ]:
# you can save the selected weights in a zip file (feature for ui)
# ExportWeights().create_zip()

In [26]:
import fiftyone as fo

In [32]:
fo.launch_app(segm_dataset.fo_dataset)

Dataset:          ImageMask
Media type:       image
Num samples:      200
Selected samples: 0
Selected labels:  0
Session URL:      http://localhost:5151/